In [ ]:
import numpy as np
import pandas as pd

The comments_to_score file has been scored offline using the PerspectiveAPI service.

In [ ]:
comments = pd.read_csv(
    '../input/perspectiveapi-scores-for-severity-ranking/perspectiveapi-scores.csv')

In [ ]:
comments.head()

In [ ]:
validation = pd.read_csv(
    '../input/jigsaw-toxic-severity-rating/validation_data.csv')

Extract the subset of the validation data that overlaps with the comments to be scored.
Note that there are many pairs of validation comments that are not part of the competition,
but they might be useful if we were training a model from scratch rather then just
using scores from an existing model.

In [ ]:
validation_annotations = validation.set_index('more_toxic').join(
    comments[['comment_id', 'text']].set_index('text'), how='inner').set_index('less_toxic').join(
    comments[['comment_id', 'text']].set_index('text'),
    lsuffix='_more', rsuffix='_less', how='inner').reset_index(drop=True)

In [ ]:
validation_annotations.head()

Perform a random search to find the best weighting of perspective scores using the overlapping validation data.

In [ ]:
best_score = 0
best_w = 0
for _ in range(40):
    w = np.random.rand(6)
    w = w / np.linalg.norm(w)
    scores =  { c[1].comment_id: sum(w * c[1][2:8]) for c in comments.iterrows() }
    percent_correct = 100.0 * (
        sum(validation_annotations['comment_id_more'].map(scores).gt(
            validation_annotations['comment_id_less'].map(scores)))
        / len(validation_annotations))
    print(percent_correct)
    if percent_correct > best_score:
        best_score = percent_correct
        best_w = w

In [ ]:
best_w

In [ ]:
best_score

In [ ]:
comments['score'] = [sum(w * c[1][2:8]) for c in comments.iterrows()]

In [ ]:
comments[['comment_id', 'score']].to_csv('submission.csv', index=False)